In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys
from devtools import debug

[sys.path.append(str(path)) for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent/"python"] if str(path) not in sys.path]

[None, None]

In [3]:
import os
import re

def replace_env_vars(string):
    return re.sub(r'\${(\w+)}', lambda match: os.environ.get(match.group(1), ''), string)

replace_env_vars("${HOME}/prj/genai-training/use_case_data")

'/home/tcl/prj/genai-training/use_case_data'